In [ ]:
import numpy as np
import torch
import rc_tools as rct
import rc_analysis as rca
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.signal import argrelextrema
import sys
import warnings
import time

import pdb

from skopt.space import Real,Integer
from skopt.utils import use_named_args
from skopt import gp_minimize

from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

import warnings

from jupyterthemes import jtplot
jtplot.style(theme='solarizedd', context='notebook', ticks=True, grid=False)

warnings.filterwarnings("ignore")
np.random.seed(11)
torch.set_printoptions(precision=10)
dtype = torch.float32 

print(f'Python: {sys.version}')
print(f'Numpy: {np.__version__}')
print(f'Torch: {torch.__version__}')

FREERUN = 20  # Extra predicition time-steps after test data
deltaT = .02      # Number of extra free-running steps  int(20/.02) 

rho = 28.0
sigma = 10.0
beta = 8/3

# Lorenz 1963
def f(state, t):
    x,y,z = state
    return sigma*(y-x), x*(rho-z)-y, x*y - beta*z

state0 = [1.,1.,1.]             # Initial point
t = np.arange(0,300+FREERUN,deltaT)  # Total of 320 full steps with deltaT=.02
states = odeint(f,state0,t)

mu = np.mean(states, axis=0)       # Get mean for each of x,y,z
signal = (states - mu)[:,[0,1,2]]  # Mean center the data
M = signal.shape[0] - int(FREERUN/deltaT)  # Length of train plus test... no freerun
#N = 25                                 # Reservoir matrix size
K = 3                                  # Input dimension
L = 3                                  # Output dimension
RF = .5                                # For feedback <--- not implemented
TEST = 1000                            # length of test
LEAD = 100                            # Number of points to plot before test
BURNIN = 100                           # Number of steps ignored for random x0 to fade
REG = 1e-8                             # Regularization factor for ridge regression
TRAINLENGTH = M-TEST    

MINMAXS = np.max(signal[:TRAINLENGTH+TEST],axis=0)-np.min(signal[:TRAINLENGTH+TEST],axis=0)
RGS = [(-19.5,19.5),(-27,27),(-25,25)]
BINS = 50

print(f'Signal length M={M}')

In [ ]:
def get_weight_matricesGPU(k,n,l,ri,ro):
    #pdb.set_trace()
    win = torch.rand((n,k),dtype=dtype,
                      device=torch.device('cuda')).sub(.5).mul(ri)
    wfb = torch.zeros((n,l),dtype=dtype, device=torch.device('cuda'))
    wout = torch.rand((l,n+k),dtype=dtype,
                      device=torch.device('cuda')).sub(.5).mul(ro)
    return win, wfb, wout

def set_vectorsGPU(n,l,r):
    #pdb.set_trace
    x0 = torch.rand((n,1),dtype=dtype,
                      device=torch.device('cuda')).sub(.5).mul(r)
    y0 = torch.zeros((l,1),dtype=dtype, device=torch.device('cuda'))
    return x0, y0

def update_res_stateGPU(wnet,xt,uxy,a,g):
    z = torch.matmul(wnet,uxy)
    return torch.mul(xt,1-a) + torch.mul(torch.tanh(z),a*g)

def predict_yGPU(wout,xu):
    return torch.matmul(wout, xu)

def get_matrixGPU(n,r,sr):
    #pdb.set_trace()
    A = (torch.rand((n,n),dtype=dtype,
                   device=torch.device('cuda'))-.5)*r
    At = torch.transpose(A,0,1)
    D = torch.diag(torch.diag(A))   
    W = A + At - D
    eig = torch.eig(W, eigenvectors=False)
    wsr = torch.max(torch.abs(eig[0]))
    return W.div(wsr).mul(sr)

def cosFactor(av,bv):
    #pdb.set_trace()
    f = 0
    cosSIM = 1.0
    try:
        currentCOS = np.dot(av, bv)/(np.linalg.norm(av)*np.linalg.norm(bv))
    except: # catch *all* exceptions
        f = sys.exc_info()[0]
        print(f)
        currentCOS = [0.0] # If error default orthogonal
        
    
    if(not f):
        if(np.any(np.isnan(currentCOS))): # If NaN default to orthogonal
            currentCOS = [0.0]
        cosSIM = 1 - currentCOS[0]
    else:
        print('Error f')
    return cosSIM

def resize_spaces(mn, mx, best):
    #pdb.set_trace()
    best_mn = np.min(best)
    best_mx = np.max(best)
    mn_bound = (best_mn-mn)/2
    mx_bound = (mx -best_mx)/2
    new_mn, new_mx = best_mn-mn_bound, best_mx+mx_bound
    print(f'Best mn:{best_mn:.3f}\t mn:{best_mx:.3f}\n')
    print(f'New bounds mn--mx: {mn_bound:.3f}--{mx_bound:.3f}')
    return new_mn, new_mx

def distribution(dataP,dataQ,mn,mx, bins=50):
    emptyFlag = []
    measureP,bnlocs = np.histogram(dataP, bins=bins, range=(mn,mx))
    measureQ,_ = np.histogram(dataQ, bins=bins, range=(mn,mx))
    for i in range(bins):
        if((measureP[i] == 0) or (measureQ[i] == 0)):
            emptyFlag.append(False)
        else:
            emptyFlag.append(True)
    #pdb.set_trace()
    measureP = measureP[emptyFlag]
    measureQ = measureQ[emptyFlag]    
    pmfP = (measureP)/np.linalg.norm(measureP)
    pmfQ = (measureQ)/np.linalg.norm(measureQ)
    kl = np.sum(rel_entr(pmfP,pmfQ))
    return (kl,pmfP,pmfQ, bnlocs)

## Training Loop for gp_minimize

In [ ]:
# Values to search with gp_optimize
#space = [Real(.5,.7, name='a'),
#         Real(.6,.8, name='sr'),
#         Real(2,4, name='amp'),
#         Real(.001,.5, name='ri'),
#         Real(2,4, name='rr')
#        ]

#space = [Real(.001, 1, name='a'),
#         Real(.001, 8., name='sr'),
#         Real(.001, 1, name='amp'), 
#         Real(.001, 1., name='ri'),
#         Real(.001, 5., name='rr')
#        ]
min_a, max_a = .001, 1.
min_sr, max_sr = .001, 8.
min_g, max_g = .001, 1.
min_ri, max_ri = .001, 1.
min_rr, max_rr = .001, 5.
space = [Real(min_a, max_a, name='a'),
                 Real(min_sr, max_sr, name='sr'),
                 Real(min_g, max_g, name='amp'), 
                 Real(min_ri, max_ri, name='ri'),
                 Real(min_rr, max_rr, name='rr')
                ]

@use_named_args(space)
def loop(a=1.0,sr=1.0,amp=1.0,ri=1.0,rr=1.0):
    start = time.time()
    global running_error, s, counter, signal, N, \
           alphas, rhos, gammas, inScales, resScales# For run-time output
    # Avoid copying from CPU to GPU    
    # Init container variables directly on GPU
    ut = torch.zeros((K,1),dtype=dtype, device=torch.device('cuda')) 
    tp = torch.zeros((K,1),dtype=dtype, device=torch.device('cuda'))
    
    # Init matrices directly on GPU
    Wres = get_matrixGPU(N,rr,sr)
    Win, Wfb, Wout = get_weight_matricesGPU(K,N,L,ri,RF)
    Wnet = torch.cat((Win,Wres,Wfb),1) # Concat to one matrix for faster compute
    xt, yt = set_vectorsGPU(N,L,rr) # On GPU random init x0 and container yt   
    
    # GPU containers for Phi and y in regression solve (bad naming... reused var name)
    # Here states is the Phi matrix
    states = torch.zeros((TRAINLENGTH, N+K),dtype=dtype, device='cuda:0')
    targets = torch.zeros((TRAINLENGTH),dtype=dtype, device='cuda:0')
    # Loop through training data and accumulate states for ridge-regression solve
    for i in range(TRAINLENGTH):
        ut[:,0] = s[i]                         # Forcing u[t] 
        tp[:,0] = s[i+1]                       # True target for prediction u[t+1] 
        uxy = torch.cat((ut,xt,yt),0)          # Concat vectors for use with Wnet
        xt1 = update_res_stateGPU(Wnet,xt,uxy,a,amp) # x[t+1] = F(x[t],u[t])
        xu = torch.transpose(torch.cat((xt1,ut),0),0,1) # Transpose as row for Phi
        states[i,:] = xu[0,:]
        xt, yt = xt1, tp 

    state = states.detach().cpu().numpy() 
    #target = targets.detach().cpu().numpy()
    
    
    ############################             Ridge Regression solve on CPU (fast!)
    torch.cuda.synchronize()    # GPU threads were running asynchronous
                                # Use signal since already sitting on CPU side
    wout = rct.get_trained_weights(state[BURNIN:],
                                   signal[BURNIN+1:TRAINLENGTH+1],
                                   REG)
                                # Move back to GPU
    Wout = torch.from_numpy(wout.reshape(L,N+K)).type(dtype).cuda() # Trained Wout
    torch.cuda.synchronize()    # Make sure synchronized before prediction pass
    ############################
    
    # Container for all predictions... Burnin --> train --> test --> freerun
    predictions = torch.zeros((M+int(FREERUN/deltaT),K),
                              dtype=dtype,
                              device=torch.device('cuda'))
    # Reset new initial vectors for prediction pass
    xt, yt = set_vectorsGPU(N,L,rr)
    ut.fill_(0.0)
    for i in range(M+int(FREERUN/deltaT)):
        if(i < TRAINLENGTH):
            ut[:,0] = s[i]
        else:
            #pdb.set_trace()
            ut = yt
        uxy = torch.cat((ut,xt,yt),0)
        xt1 = update_res_stateGPU(Wnet,xt,uxy,a,amp)
        xu = torch.cat((xt1,ut),0)
        yt1 = predict_yGPU(Wout,xu)
        #pdb.set_trace()
        predictions[i] = yt1[:,0]
        xt, yt = xt1, yt1

    yHat_GPU = predictions.detach().cpu().numpy()  # Move predictions onto CPU (numpy)
    
    nrmse = np.ones((K,1))*1000
    #if((np.any(np.isnan(yHat_GPU))) or (np.any(np.isinf(yHat_GPU)))):
    #    nrmse = nrmse_fail
    #else:
    try:
        for i in range(K):
            nrmse[i,0] = rca.NRMSE(signal[TRAINLENGTH:TRAINLENGTH+TEST,i],
                                   yHat_GPU[TRAINLENGTH:TRAINLENGTH+TEST,i],
                                   MINMAXS[i])
    except: 
        pass
    #### Pearson Correlation <=> Cosine Distance    
    #pdb.set_trace()
    av = signal[TRAINLENGTH:TRAINLENGTH+TEST]
    bv = np.squeeze(yHat_GPU[TRAINLENGTH:TRAINLENGTH+TEST])
    dists = np.zeros(K)
    for i in range(K):
        avec = av[:,i].reshape(TEST,1)
        bvec = bv[:,i].reshape(TEST,1)
        num = np.squeeze(np.dot(avec.T,bvec))
        den = np.linalg.norm(avec)*np.linalg.norm(bvec)
        cosine_similarity = num/den
        cosine_distance = 1 - cosine_similarity
        dists[i] = cosine_distance
        
    loss = np.max(nrmse+dists)
    if(np.isnan(loss) or (np.isinf(loss) or (loss > 1000.0))):
        loss = 1000
    
    if(loss < running_error):
        alphas.append(a)
        rhos.append(sr)
        gammas.append(amp)
        inScales.append(ri)
        resScales.append(rr)
        running_error = loss
        #wnet = Wnet.detach().cpu().numpy()
        
        plt.figure(figsize=(15,12))
        plt.subplot(3,1,1)
        plt.plot(signal[TRAINLENGTH-LEAD:,0], label='Target')
        plt.plot(yHat_GPU[TRAINLENGTH-LEAD:,0], label='X pred')
        plt.axvline(LEAD,c='r',linestyle='dashed')
        plt.ylim(-20,20)
        plt.legend()
        
        plt.subplot(3,1,2)
        plt.plot(signal[TRAINLENGTH-LEAD:,1], label='Target')
        plt.plot(yHat_GPU[TRAINLENGTH-LEAD:,1], label='Y pred')
        plt.axvline(LEAD,c='r',linestyle='dashed')
        plt.ylim(-20,20)
        plt.legend()
        
        plt.subplot(3,1,3)
        plt.plot(signal[TRAINLENGTH-LEAD:,2], label='Target')
        plt.plot(yHat_GPU[TRAINLENGTH-LEAD:,2], label='Z pred')
        plt.axvline(LEAD,c='r',linestyle='dashed')
        plt.ylim(-20,20)
        plt.legend()
        
        plt.show()
        plt.close()
        
        #np.save(f'./MultiModelL3D/Mats/L3D_it{counter}_{N}_Wnet',wnet)
        #np.save(f'./MultiModelL3D/Preds/L3D_it{counter}_{N}_Preds',yHat_GPU)
        #np.save(f'./MultiModelL3D/Params/L3D_it{counter}_{N}_InstanceParams',currParams)
        print(f' Iter={counter} a={a:.3f} sr={sr:.3f} amp={amp:.3f}',
              f' ri={ri:.3f} rr={rr:.3f} loss={loss:3f}\n\n')
    print(f'Iter: {counter} #### Diagnostic {loss:3f}   Time {(time.time()-start):.2f}',
          f' Best {running_error:.3f} NRMSE {np.max(nrmse):.3f} CD {np.max(dists):.3f}')
    counter += 1
    return loss

## Parameter Search with gp_minimize

In [ ]:

CALLS = 100      
s = torch.torch.from_numpy(signal).cuda().type(dtype)
# 
size = [1000,950,900,850,800,750,700,650,600,550,500,
        450,400,350,300,250,200,150,100,50]
for ref in range(50):
    start = time.time()
    for N in [100]:
        alphas = []
        rhos = []
        gammas = []
        inScales = []
        resScales = []
        space = [Real(min_a, max_a, name='a'),
                 Real(min_sr, max_sr, name='sr'),
                 Real(min_g, max_g, name='amp'), 
                 Real(min_ri, max_ri, name='ri'),
                 Real(min_rr, max_rr, name='rr')
                ]
        running_error = 1000
        counter = 0 
        print(f'********** {N} ***********')
        result_gp = gp_minimize(loop, space, n_calls=CALLS,
                                random_state=11, n_jobs=2, n_initial_points=100)
        print(f'\nBest result = {result_gp.fun}')
        names = ['a','sr','amp','ri','rr']
        for i in range(len(space)):
            print(f'{names[i]} = {result_gp.x[i]}')
        min_a, max_a   = resize_spaces(min_a, max_a, np.array(alphas))
        min_sr, max_sr = resize_spaces(min_sr, max_sr, np.array(rhos))
        min_g, max_g   = resize_spaces(min_g, max_g, np.array(gammas))
        min_ri, max_ri = resize_spaces(min_ri, max_ri, np.array(inScales))
        min_rr, max_rr = resize_spaces(min_rr, max_rr, np.array(resScales))
        print('Refined search bounds:\n')
        print(f'Alpha ({min_a}, {max_a})\n')
        print(f'Rho ({min_sr}, {max_sr})\n')
        print(f'Gamma ({min_g}, {max_g})\n')
        print(f'r-in ({min_ri}, {max_ri})\n')
        print(f'r-res ({min_rr}, {max_rr})\n')
    end = time.time()-start
    print(f'End Refinement Run {ref} Time {end:.3f}')

## MultiModel RC Analysis

In [ ]:
def rank_curve(cplus, tols):
    rank_tols = []
    for i in tols:
        rank_tols.append(rca.rank(cplus, i))
    return np.array(rank_tols)

iteration = [149,135,63,14,67,38,128,140,38,63,119,7,86,28,149,102,49,68, 17,17]
size = [1000,950,900,850,800,750,700,650,600,550,500,450,400,350,300,250,200,150,100,50]

TOLS = [1/10**(x) for x in range(0,20)]
mm_ranks = []
for ctr,N in zip(iteration,size):
    start = time.time()
    fn_mats = f'./MultiModelL3D/Mats/L3D_it{ctr}_{N}_Wnet.npy'
    fn_params = f'./MultiModelL3D/Params/L3D_it{ctr}_{N}_InstanceParams.npy'
    p = np.load(fn_params, allow_pickle=True)
    a,g = p[0],p[2]

    Wr, Wi = rca.get_mats(fn_mats, 3, N)
    xeq = np.zeros((N,1))
    ueq = np.zeros((K,1))
    A = rca.leaky_jacobian(xeq, ueq, a, g, Wi, Wr)
    B = rca.partial_u(xeq, ueq, a, g, Wi, Wr)

    Cplus = rca.reachable_matrix(A, B)
    svs = np.linalg.svd(Cplus, compute_uv=False)
    Cplus = Cplus/np.max(svs)
    rkc = rank_curve(Cplus,TOLS)
    v = np.argmax(np.gradient(rkc))-1
    ave_rank = (rkc[v]+rkc[v+1])/2
    print(f'Time {time.time()-start} Pair: iter {ctr} size {N} argmax {v} {v+1} tol {TOLS[v]} ave {ave_rank}')
    mm_ranks.append(ave_rank)
    
np.save('./MultiModelL3D/L3D_MMranks',mm_ranks)

In [ ]:
minmax = np.max(signal[:TRAINLENGTH+TEST,:], axis=0)-np.min(signal[:TRAINLENGTH+TEST,:], axis=0)
err_list = []
for ctr,N in zip(iteration,size):
    pred = np.load(f'./MultiModelL3D/Preds/L3D_it{ctr}_{N}_Preds.npy', allow_pickle=True)
    nrmse = mean_squared_error(signal[TRAINLENGTH:TRAINLENGTH+TEST,:],pred[TRAINLENGTH:TRAINLENGTH+TEST,:],
                               squared=False, multioutput='raw_values')
    print(nrmse/minmax)
    err_list.append(nrmse/minmax)
np.save('./MultiModelL3D/L3D_mm_errors',np.array(err_list))

plt.plot(err_list)
plt.show()

In [ ]:
plt.plot(mm_ranks)
plt.show()

In [ ]:
np.mean(np.array(mm_ranks))

In [ ]:
np.std(np.array(mm_ranks))

In [ ]:
total = 0.
no_outliers = []
for i in range(len(mm_ranks)):
    if(i in [4,13,17]):
        pass
    else:
        total += mm_ranks[i]
        no_outliers.append(mm_ranks[i])
total/(len(mm_ranks)-3)

In [ ]:
no =np.array(no_outliers) 
np.mean(no)

In [ ]:
np.std(no)

In [ ]:
minmax.shape